The purpose of this notebook is to build sklearn-like pipeline for transformation

In [22]:
import pandas as pd         
import os.path

N_JOBS = 6
DEBUG=True

In [23]:
def load_data(path="../../data/csv/all_v2.csv"):
    dataframe = pd.read_csv(path, index_col=0)
    return dataframe.loc[~dataframe["execTimeMs"].isnull()]

In [24]:
def prepare_dataframe(dataframe):
    output = dataframe.dropna(axis="columns")
    targets = output["execTimeMs"]
    dropped = output[["command", "execTimeMs", "jobId", "ctime_mean", "ctime_max", "ctime_sum", "read_sum","write_sum","readSyscalls_sum","writeSyscalls_sum","readReal_sum","writeReal_sum","writeCancelled_sum","rxBytes_sum","rxPackets_sum","rxErrors_sum","rxDrop_sum","rxFifo_sum","rxFrame_sum","rxCompressed_sum","rxMulticast_sum","txBytes_sum","txPackets_sum","txErrors_sum","txDrop_sum","txFifo_sum","txColls_sum","txCarrier_sum","txCompressed_sum","cpu_mean","cpu_max","memory_mean","memory_max"]]
    features = output.drop(dropped.columns, axis=1)
    return features, targets, dropped

In [25]:
features, targets, dropped = prepare_dataframe(load_data())

In [26]:
features.dtypes

workflowName              object
size                     float64
executable                object
args                      object
inputs                    object
outputs                   object
name                      object
cpu.manufacturer          object
cpu.brand                 object
cpu.speed                float64
cpu.cores                  int64
cpu.physicalCores          int64
cpu.processors             int64
mem.total                  int64
mem.free                   int64
mem.used                   int64
mem.active                 int64
mem.available              int64
mem.buffers                int64
mem.cached                 int64
mem.slab                   int64
mem.buffcache              int64
mem.swaptotal              int64
mem.swapused               int64
mem.swapfree               int64
total_cpus               float64
avg_cpus                 float64
avg_pods                 float64
total_ram_available        int64
average_ram_available    float64
dtype: obj

# Preprocessing flow

In [27]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector as selector

In [28]:
def vectorize_list(series):
    def vectorize(list_string):
        return len(eval(list_string))
    return np.vectorize(vectorize)(series)

def ListTransformer():
    return FunctionTransformer(func=vectorize_list)

In [29]:
list_transformer = Pipeline(steps=[("list", ListTransformer()), ("scaler", StandardScaler())])
list_features = list(['args', 'inputs', 'outputs'])

numerical_transformer = StandardScaler()
numerical_features = list(features.select_dtypes(include="number").columns)

categorical_transformer = OneHotEncoder(sparse=False, handle_unknown = "ignore")
categorical_features = list(set(features.select_dtypes(include="object").columns) ^ set(list_features))

def make_classifying_preprocessor(additional_features=["read_sum", "write_sum", "cpu_mean", "memory_max"]):
    external_features = categorical_features + additional_features
    return ColumnTransformer(
            transformers=[('lists', list_transformer, list_features), 
                          ('num', numerical_transformer, numerical_features),
                          ('cat', categorical_transformer, external_features)])

def make_regression_preprocessor(additional_features=["read_sum", "write_sum", "cpu_mean", "memory_max"]):
    external_features = numerical_features + additional_features
    return ColumnTransformer(
        transformers=[
            ('lists', list_transformer, list_features),            
            ('num', numerical_transformer, external_features),  
            ('cat', categorical_transformer, categorical_features)
        ])

preprocessor = make_classifying_preprocessor(additional_features=[])

In [30]:
from scipy.stats import percentileofscore
import math

def calculate_quantile_rank(labels, label):
    return percentileofscore(labels, label) / 100

def calculate_utilization_class(labels, label):
    def label_for_rank(rank):
        if rank > 0.75:
            return 'very high'
        elif rank > 0.5:
            return 'high'
        elif rank > 0.25:
            return 'medium'
        else:
            return 'low'
    return label_for_rank(calculate_quantile_rank(labels, label))

def calculate_utilization_bucket(labels, label, num_buckets):
    bucket_size = 1.0 / num_buckets
    def bucket_for_rank(rank):
        return str(math.floor(rank / bucket_size))
    return bucket_for_rank(calculate_quantile_rank(labels, label))

# Pipeline composition (with PCA)

In [31]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.linear_model import Lasso, SGDRegressor, ElasticNet, LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.dummy import DummyRegressor
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, GridSearchCV, HalvingGridSearchCV
from sklearn.kernel_ridge import KernelRidge

In [32]:
base_steps = [('pca', PCA(random_state=42))]
dummy_pipeline = Pipeline(steps=base_steps +[('dummy', DummyRegressor())])
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=0)

In [33]:
pca_param_grid = {
    'pca__n_components': np.arange(1, 50, 1),    
}
knn_param_grid = {
    'knn__n_neighbors': np.arange(1, 30, 3),
}
regressor = ('knn', KNeighborsRegressor())
full_pipeline = Pipeline(steps= base_steps + [regressor])
grid_search = HalvingGridSearchCV(full_pipeline, {**knn_param_grid, **pca_param_grid}, cv=2, verbose=2, scoring="r2", n_jobs=-1)

In [34]:
def rae(actual, predicted):
    """ Relative Absolute Error (aka Approximation Error) """
    EPSILON=1e-10
    return np.sum(np.abs(actual - predicted)) / (np.sum(np.abs(actual - np.mean(actual))) + EPSILON)

In [35]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, make_scorer

In [36]:
rae_scorer=make_scorer(rae, greater_is_better=False)

In [37]:
def calculate_regression_score(true, pred, scores=[r2_score, mean_absolute_error, mean_absolute_percentage_error, rae]):
    executor = get_reusable_executor(max_workers=4)
    results = executor.map(lambda fun: fun(true, pred), scores)
    return results

In [134]:
from loky import get_reusable_executor

def rate_regressor(X_train, y_train, X_test, y_test, regressor, regressor_params, verbose=10, aggressive_elimination=True, steps=base_steps, scoring="r2"):
    if DEBUG:
        print(f"Rating {regressor}")
    full_pipeline = Pipeline(steps= base_steps + [regressor])
    vector_length = min(X_train.shape[0], X_train.shape[1])
    pca_param_grid = {'pca__n_components': np.arange(1, vector_length, 1),}
    grid_search = HalvingGridSearchCV(full_pipeline, {**pca_param_grid, **regressor_params}, cv=2, verbose=verbose, scoring=scoring, n_jobs=N_JOBS)
    if DEBUG:
        print("Evaluating grid search")
    grid_search.fit(X_train, y_train)
    
    # scores
    if DEBUG:
        print("Predicting on test set")
    prediction = grid_search.best_estimator_.predict(X_test)
    
    if DEBUG:
        print("Calculating scores")
    r2, mae, mape, rae = calculate_regression_score(y_test, prediction)
    if DEBUG:
        print("Calculated scores on test set")
    adjusted_r2 = 1 - (1-r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
    return {"r2": r2, "adjusted_r2": adjusted_r2, "mae": mae, "mape": mape, "rae": rae,"best_score": grid_search.best_score_, "params": grid_search.best_params_}

# Here go regressor params

In [135]:
knn = ("knn", KNeighborsRegressor())
knn_params = {'knn__n_neighbors': np.arange(1, 30, 1)}

lasso = ("lasso", Lasso(random_state=5))
lasso_params = {"lasso__alpha": np.arange(0.01, 1, 0.05)}

mlp_regressor = ("mlp", MLPRegressor(random_state=5))
mlp_params = {"mlp__activation": ["relu", "logistic"], "mlp__hidden_layer_sizes": [(100,), (100, 50,)]}

dtr = ("dtr", DecisionTreeRegressor(random_state=5))
dtr_params = {"dtr__criterion": ["mse", "friedman_mse", "mae", "poisson"], "dtr__max_depth": [5, 10, 15, 25]}

en = ("elasticnet", ElasticNet(random_state=5))
en_params = {"elasticnet__alpha": np.arange(0.01, 1, 0.05), "elasticnet__l1_ratio": np.arange(0, 1, 0.1)}

svr = ("svr", SGDRegressor())
svr_params = {"svr__loss": ["squared_loss", "huber", "epsilon_insensitive"], "svr__penalty": ['l2', 'l1', 'elasticnet'],
             "svr__alpha": np.arange(0.0001, 0.2, 0.01), "svr__max_iter": [10000]}

In [136]:
import warnings
warnings.filterwarnings('ignore')

In [137]:
def rate_data(features, targets, regressors, verbose=10, pipeline_steps=base_steps):
    X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, random_state=0)
    df = pd.DataFrame(columns=["name", "pca", "adjusted_r2","r2", "mae", "mape", "rae", "best_score", "params"])
    for (regressor, params) in regressors:
        result = rate_regressor(X_train, y_train, X_test, y_test, regressor, params, verbose, pipeline_steps)
        df = df.append({"name": regressor[0], **result, "pca": result["params"]["pca__n_components"]}, ignore_index=True)
    return df

In [138]:
def rate_data_explicit(X_train, X_test, y_train, y_test, regressors, verbose=10, pipeline_steps=base_steps, scoring="r2"):
    df = pd.DataFrame(columns=["name", "pca", "adjusted_r2","r2", "mae", "mape", "best_score", "params"])
    for (regressor, params) in regressors:
        result = rate_regressor(X_train, y_train, X_test, y_test, regressor, params, verbose, pipeline_steps, scoring=scoring)
        df = df.append({"name": regressor[0], **result, "pca": result["params"]["pca__n_components"]}, ignore_index=True)
    return df

In [139]:
def rate_classifiers_for_data(features, targets, classifiers, verbose=10, pipeline_steps=base_steps):
    X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, random_state=0)
    df = pd.DataFrame(columns=["name", "pca", "accuracy","balanced_accuracy", "f1_micro", "f1_macro", "params"])
    for (classifier, params) in classifiers:
        result = rate_classifier(X_train, y_train, X_test, y_test, classifier, params, verbose, pipeline_steps)
        df = df.append({"name": classifier[0], **result, "pca": result["params"]["pca__n_components"]}, ignore_index=True)
    return df

In [140]:
def rate_dataset(dataframe, regressors, verbose=2):
    print(f"Rating dataset of len {len(dataframe)}")
    features, targets, _ = prepare_dataframe(dataframe[:10000])
    features = preprocessor.fit_transform(features)
    rate_data(features, targets, regressors, verbose)

In [141]:
basic_regressors = [
    (knn, knn_params),
    (dtr, dtr_params),
    (lasso, lasso_params),
    (mlp_regressor, mlp_params),
    (en, en_params),
    (svr, svr_params),
]

In [142]:
def simple_experiment():
    print("Rating jobs datasets")
    for dataset in dfs_for_jobs:
        print(dataset.iloc[0]["name"])
        rate_dataset(dataset, basic_regressors)

    print("Rating common datasets")
    for dataset in datasets:
        rate_dataset(dataset, basic_regressors)

In [143]:
# rate_dataset(dfs_for_jobs[8], verbose=0)
# rate_dataset(dfs_for_jobs[1], basic_regressors, verbose=0)

## Eksperyment 4

### Cel

Zmierzyć skuteczności najlepszych pipelinów dla każdego joba, zobaczyć czy warto schodzić w dół pod wzgledem błędów

### Dane

In [144]:
full_df = load_data().dropna(axis="columns")
raw_datasets = { x:pd.DataFrame(y) for x, y in full_df.groupby('name', as_index=False)}
datasets_split = {x:train_test_split(df, random_state=0, train_size=0.75) for x,df in raw_datasets.items()}

In [145]:
def get_numerical_pipeline_data_big(data, resources=["read_sum", "write_sum", "cpu_mean", "memory_max"]):
    """
    Raw data enhanced with resource utilization quantile scores, but scores are assigned - not predicted
    """
    features, labels, dropped = prepare_dataframe(data)
    for resource in resources:
        features[resource] = dropped[resource].map(lambda value: calculate_quantile_rank(dropped[resource], value))
    features = make_regression_preprocessor(resources).fit_transform(features)
    return pd.DataFrame(features, index=labels.index), labels

In [146]:
def get_categorical_pipeline_data_big(data, resources=["read_sum", "write_sum", "cpu_mean", "memory_max"]):
    features, labels, dropped = prepare_dataframe(data)
    for resource in resources:
        features[resource] = dropped[resource].map(lambda value: calculate_utilization_bucket(dropped[resource], value, num_buckets=8))
    features = make_classifying_preprocessor(resources).fit_transform(features)
    return pd.DataFrame(features, index=labels.index), labels

In [147]:
datasets_split['add_replace'][1]

,jobId,read_sum,write_sum,readSyscalls_sum,writeSyscalls_sum,readReal_sum,writeReal_sum,writeCancelled_sum,rxBytes_sum,rxPackets_sum,...,mem.slab,mem.buffcache,mem.swaptotal,mem.swapused,mem.swapfree,total_cpus,avg_cpus,avg_pods,total_ram_available,average_ram_available
74622,o5B3JL1c0-1-60,393064.0,9.0,289.0,9.0,0.0,32768.0,0.0,0.0,0.0,...,509407232,4113100800,0,0,0,13.69,3.422500,110.000000,47131832000,1.178296e+10
39958,VdebhcHGz-1-86,423392.0,9.0,317.0,9.0,0.0,32768.0,0.0,13186.0,42.0,...,836173824,4935049216,0,0,0,19.60,3.920000,110.000000,68207180000,1.364144e+10
18723,6sXqyD-cD-1-372,382827.0,9.0,280.0,9.0,0.0,32768.0,0.0,0.0,0.0,...,705224704,10695143424,0,0,0,19.60,3.920000,110.000000,68207180000,1.364144e+10
81198,uwu6POi40-1-86,436311.0,9.0,335.0,9.0,0.0,32768.0,0.0,0.0,0.0,...,317255680,2782396416,0,0,0,23.52,3.920000,110.000000,81848584000,1.364143e+10
6678,1XRbCZX-m-1-34,429309.0,9.0,326.0,9.0,0.0,32768.0,0.0,0.0,0.0,...,317988864,5578797056,0,0,0,15.68,3.920000,110.000000,54565728000,1.364143e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28118,yXB4dcCCr-1-112,480781.0,9.0,437.0,9.0,49152.0,32768.0,0.0,11340.0,27.0,...,206446592,3237965824,0,0,0,7.72,1.930000,30.500000,25340644000,6.335161e+09
14761,Qt1V_tSyk-1-138,244818.0,9.0,145.0,9.0,0.0,32768.0,0.0,11586.0,28.0,...,1794420736,8417120256,0,0,0,17.75,5.916667,44.333333,66543572000,2.218119e+10
53990,ddDQIVS8y-1-502,461037.0,19.0,377.0,10.0,0.0,32768.0,0.0,0.0,0.0,...,329924608,3229777920,0,0,0,23.52,3.920000,110.000000,81848616000,1.364144e+10
74749,oXReaKxEP-1-138,398153.0,9.0,298.0,9.0,0.0,32768.0,0.0,12266.0,32.0,...,602771456,8635564032,0,0,0,19.60,3.920000,110.000000,68207180000,1.364144e+10


### Przebieg


trenujemy pipeline ogólny, liczymy jego skuteczności dla każdego typu jobów. Z eksperymentu 1. - najlepszy pipeline to był:

dtr, mae, pca 71

dla każdego typu jobów trenujemy dla niego pipeline, liczymy skuteczności

In [148]:
train, test = datasets_split['add_replace']

In [149]:
def run_experiment4():
    exp4_resources = ["read_sum", "write_sum", "cpu_max", "cpu_mean", "memory_mean", "memory_max"]
        
    big_regressor = Pipeline([ # test pipeline
        ('pca', PCA(random_state=42, n_components=71)),
#         ('knn', KNeighborsRegressor(n_neighbors=11))
        ('dtr', DecisionTreeRegressor(criterion="mae"))
    ])
    print("Preparing data for big regressor")
    X, y = get_numerical_pipeline_data_big(full_df, exp4_resources)
    train_indices = np.concatenate([train.index for (_, (train, _)) in datasets_split.items()])
    test_indices = np.concatenate([test.index for (_, (_, test)) in datasets_split.items()])
    print(f"{len(X)} {len(y)} {len(train_indices)} {len(test_indices)}")
    X_train, X_test, y_train, y_test = X.loc[train_indices], X.loc[test_indices], y.loc[train_indices], y.loc[test_indices]
    print(f"Making split with test as {len(test_indices)/(len(test_indices) + len(train_indices))} of dataset")
    
    print("Training big regressor with train data")
    big_regressor.fit(X_train, y_train)
    
    print("Predicting with big regressor on test data")
    y_predicted = big_regressor.predict(X_test)
    
    print("Rating big regressor's overall performance")
    [r2, mae, mape, rae_score] = calculate_regression_score(y_test, y_predicted, [r2_score, mean_absolute_error, mean_absolute_percentage_error, rae])
    print(f"Scores for big regressor:")
    print(f"R2: {r2}")
    print(f"MAE: {mae}")
    print(f"MAPE: {mape}")
    print(f"RAE: {rae_score}")
    print()
    dataframes = []
    for (job, (train, test)) in datasets_split.items():
        print(f"Comparing big regressor vs local regressor for job {job}")
        print("Preparing data for local regressor")
        joint_df = pd.concat([train, test])
        local_X, local_y = get_numerical_pipeline_data_big(joint_df, exp4_resources)
        print(f"{len(local_X)} {len(train.index)} {len(test.index)}")
        local_X_train, local_X_test, local_y_train, local_y_test = local_X.loc[train.index], local_X.loc[test.index], local_y.loc[train.index], local_y.loc[test.index]
        
        print(f"Rating local regressors for job {job}")
        regressor_df = rate_data_explicit(local_X_train, local_X_test, local_y_train, local_y_test, basic_regressors, verbose=0)
        print(regressor_df.head())
        print("Preparing data for big regressor")
        local_X_test, local_y_test = X.loc[test.index], y.loc[test.index]
        
        print(f"Rating big regressor for job {job}")
        local_predicted = big_regressor.predict(local_X_test)
        [r2, mae, mape, rae_score] = calculate_regression_score(local_predicted, local_y_test, [r2_score, mean_absolute_error, mean_absolute_percentage_error, rae])
        regressor_df = regressor_df.append({"name": "big", **{"r2": r2, "mae": mae, "mape": mape, "rae": rae_score}, "pca": "xD"}, ignore_index=True)
        regressor_df["job"] = job
        regressor_df["size"] = len(joint_df)
        dataframes.append(regressor_df)
        
    return pd.concat(dataframes)

In [150]:
# def run_experiment4_memory_intensive():
#     exp4_resources = ["read_sum", "write_sum", "cpu_max", "cpu_mean", "memory_mean", "memory_max"]
#     train_indices = np.concatenate([train.index for (_, (train, _)) in datasets_split.items()])
#     test_indices = np.concatenate([test.index for (_, (_, test)) in datasets_split.items()])
#     dataframes = []
#     for (job, (train, test)) in datasets_split.items():
#         global N_JOBS
#         print("Preparing data for local regressor")
#         joint_df = pd.concat([train, test])
#         local_X, local_y = get_numerical_pipeline_data_big(joint_df, exp4_resources)
#         print(f"{len(local_X)} {len(train.index)} {len(test.index)}")
        
#         local_X_train, local_X_test, local_y_train, local_y_test = local_X.loc[train.index], local_X.loc[test.index], local_y.loc[train.index], local_y.loc[test.index]
#         if len(local_X) > 40_000:
#             N_JOBS=1
#             local_X_train = local_X_train.take(10_000)
#             local_y_train = local_y_train.take(10_000)
#             size = 10_0000
#         else:
#             size = len(joint_df)
        
#         print(f"Rating local regressors for job {job}")
#         regressor_df = rate_data_explicit(local_X_train, local_X_test, local_y_train, local_y_test, memory_intensive_regressors, verbose=0)
#         print(regressor_df.head())
#         regressor_df["job"] = job
#         regressor_df["size"] = size
#         dataframes.append(regressor_df)
        
#         N_JOBS=7
#     return pd.concat(dataframes)

In [151]:
# def run_experiment4_categorical():
#     exp4_resources = ["read_sum", "write_sum", "cpu_max", "cpu_mean", "memory_mean", "memory_max"]
#     train_indices = np.concatenate([train.index for (_, (train, _)) in datasets_split.items()])
#     test_indices = np.concatenate([test.index for (_, (_, test)) in datasets_split.items()])
#     dataframes = []
#     for (job, (train, test)) in datasets_split.items():
#         print("Preparing data for local regressor")
#         joint_df = pd.concat([train, test])
#         local_X, local_y = get_categorical_pipeline_data_big(joint_df, exp4_resources)
#         print(f"{len(local_X)} {len(train.index)} {len(test.index)}")
#         local_X_train, local_X_test, local_y_train, local_y_test = local_X.loc[train.index], local_X.loc[test.index], local_y.loc[train.index], local_y.loc[test.index]
        
#         print(f"Rating local regressors for job {job}")
#         regressor_df = rate_data_explicit(local_X_train, local_X_test, local_y_train, local_y_test, basic_regressors, verbose=0)
#         print(regressor_df.head())
#         regressor_df["job"] = job
#         regressor_df["size"] = len(joint_df)
#         dataframes.append(regressor_df)
        
#     return pd.concat(dataframes)

In [156]:
def run_experiment4_numerical_rae():
    exp4_resources = ["read_sum", "write_sum", "cpu_max", "cpu_mean", "memory_mean", "memory_max"]
        
    
    train_indices = np.concatenate([train.index for (_, (train, _)) in datasets_split.items()])
    test_indices = np.concatenate([test.index for (_, (_, test)) in datasets_split.items()])
    dataframes = []
    for (job, (train, test)) in datasets_split.items():
        print("Preparing data for local regressor")
        joint_df = pd.concat([train, test])
        local_X, local_y = get_numerical_pipeline_data_big(joint_df, exp4_resources)
        print(f"{len(local_X)} {len(train.index)} {len(test.index)}")
        local_X_train, local_X_test, local_y_train, local_y_test = local_X.loc[train.index], local_X.loc[test.index], local_y.loc[train.index], local_y.loc[test.index]
        
        print(f"Rating local regressors for job {job}")
        regressor_df = rate_data_explicit(local_X_train, local_X_test, local_y_train, local_y_test, basic_regressors, verbose=0, scoring=rae_scorer)
        print(regressor_df.head())
        regressor_df["job"] = job
        regressor_df["size"] = len(joint_df)
        dataframes.append(regressor_df)
        
    return pd.concat(dataframes)

In [157]:
def load_or_run(file, runner):
    if not os.path.isfile(file):
        print(f"Running experiment {file}")
        dataframe = runner()
        dataframe.to_csv(file)
    else:
        dataframe = pd.read_csv(file).round(2)
    return dataframe

In [158]:
N_JOBS=8

In [159]:
exp4_r2_df_incomplete = load_or_run("data/exp4_optimize_r2.csv", run_experiment4)
exp4_rae_incomplete = load_or_run("data/exp4_optimize_rae.csv", run_experiment4_numerical_rae)
# exp4_r2_df_memory_intensive = load_or_run("data/exp4_optimize_r2_memint.csv", run_experiment4_memory_intensive)
# exp4_r2_categorical_incomplete = load_or_run("data/exp4_optimize_r2_categorical.csv", run_experiment4_categorical)

Running experiment data/exp4_optimize_rae.csv
Preparing data for local regressor
427 320 107
Rating local regressors for job add_replace
Rating ('knn', KNeighborsRegressor())
Evaluating grid search
Predicting on test set
Calculating scores
Calculated scores on test set
Rating ('dtr', DecisionTreeRegressor(random_state=5))
Evaluating grid search
Predicting on test set
Calculating scores
Calculated scores on test set
Rating ('lasso', Lasso(random_state=5))
Evaluating grid search
Predicting on test set
Calculating scores
Calculated scores on test set
Rating ('mlp', MLPRegressor(random_state=5))
Evaluating grid search
Predicting on test set
Calculating scores
Calculated scores on test set
Rating ('elasticnet', ElasticNet(random_state=5))
Evaluating grid search
Predicting on test set
Calculating scores
Calculated scores on test set
Rating ('svr', SGDRegressor())
Evaluating grid search
Predicting on test set
Calculating scores
Calculated scores on test set
         name pca  adjusted_r2     

In [ ]:
exp4_r2_df_incomplete.sort_values('rae', ascending=True).drop_duplicates(['job'])

### Wyniki

Wykresy:
- najlepsza skuteczność regresora (min mape/rae) vs liczba sampli
- skuteczności 5 najlepszych regresorów dla każdego (grid chart)
- najlepsza skuteczność regresora dla joba vs skuteczność dużego regresora dla tego joba

Odpowiedzi:
- czy zwiększenie granularności jest sensowne?
- czy jest jakiś widoczny próg liczby sampli przy zwiększonej granularności?